# Langchain integration

### LLMstudio setup

In [ ]:
from llmstudio.llm.langchain import ChatLLMstudio
from llmstudio import LLM

In [ ]:
# llm = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)
llm = ChatLLMstudio(model_id='vertexai/gemini-1.5-flash', temperature=0)

### Langchain setup

In [ ]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

In [ ]:
llm.invoke('Hello')

### Example 1: Train ticket

In [ ]:
@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"


def assistant(question: str)->str:
    tools = [get_departure, buy_ticket]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [ ]:
assistant('When does my train depart? My ticket is 1234')


In [ ]:
assistant('Buy me a ticket to Madrid and tell the departure time')

### Example 2: Start a party

In [ ]:
@tool
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

@tool
def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

@tool
def dim_lights(brightness: float) -> bool:
    """Dim the lights.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True


In [ ]:
def assistant(question: str)->str:
    tools = [power_disco_ball, start_music, dim_lights]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [ ]:
assistant('Turn this into a party!')